In [3]:
import numpy as np
import pandas as pd 
from itertools import izip

Offensive Rating is defined as the team points scored per 100 possessions while the player is on the court.

Defensive Rating is defined as the number of points per 100 possessions that the team alows while that individual player is on the court. 

A possession is ended by
 -1 made field goal attempts
 -2 Made final free throw attempt
 -3 Missed final free throw attempt that results in a defensive reboud
 -4 Missed field goal attempt that resutls in a defensive rebound
 -5 turnover
 -6 end of time period

In [300]:
EventCodes = pd.read_csv('Event_Codes.txt', sep='\t')
#Strip excess whitespace that I noticed in Event Message Type Description
EventCodes['Event_Msg_Type_Description'] = EventCodes['Event_Msg_Type_Description'].map(str.strip)
#eventCodes['Action_Type_Description'] = eventCodes['Action_Type_Description'].map(str.strip)

GameLineup = pd.read_csv('Game_Lineup.txt', sep='\t')
PlayByPlay = pd.read_csv('Play_by_Play.txt', sep='\t')

In [5]:
from collections import defaultdict

In [428]:
EventScoringValues = defaultdict(dict) #the point value of a play, for the offensive team
EventPossChange= defaultdict(dict) #whether the team could constitute a change of possesion
EventSub = defaultdict(dict)

In [429]:
for row in EventCodes.iterrows():
    #initialize
    e_type, a_type = row[1]['Event_Msg_Type'], row[1]['Action_Type']
    EventScoringValues[e_type][a_type] = 0
    EventPossChange[e_type][a_type] = False
    EventSub[e_type][a_type] = False

In [430]:
_,idx1 = np.unique(EventCodes['Event_Msg_Type'], return_index=True)
_,idx2 = np.unique(EventCodes['Action_Type'], return_index=True)

In [431]:
idx = np.array(list(set(idx1).union(idx2)))
idx.sort()

In [432]:
for i in idx:
    print EventCodes['Event_Msg_Type'][i], EventCodes['Event_Msg_Type_Description'][i],EventCodes['Action_Type'][i], EventCodes['Action_Type_Description'][i]

1 Made Shot 0 No Shot
1 Made Shot 1 Jump Shot
1 Made Shot 2 Running Jump Shot
1 Made Shot 3 Hook Shot
1 Made Shot 4 Tip Shot
1 Made Shot 5 Layup Shot
1 Made Shot 6 Driving Layup Shot
1 Made Shot 7 Dunk Shot
1 Made Shot 8 Slam Dunk Shot
1 Made Shot 9 Driving Dunk Shot
1 Made Shot 40 Layup Shot
1 Made Shot 41 Running Layup Shot
1 Made Shot 42 Driving Layup Shot
1 Made Shot 43 Alley Oop Layup shot
1 Made Shot 44 Reverse Layup Shot
1 Made Shot 45 Jump Shot
1 Made Shot 46 Running Jump Shot
1 Made Shot 47 Turnaround Jump Shot
1 Made Shot 48 Dunk Shot
1 Made Shot 49 Driving Dunk Shot
1 Made Shot 50 Running Dunk Shot
1 Made Shot 51 Reverse Dunk Shot
1 Made Shot 52 Alley Oop Dunk Shot
1 Made Shot 53 Tip Shot
1 Made Shot 54 Running Tip Shot
1 Made Shot 55 Hook Shot
1 Made Shot 56 Running Hook Shot
1 Made Shot 57 Driving Hook Shot
1 Made Shot 58 Turnaround Hook Shot
1 Made Shot 59 Finger Roll Shot
1 Made Shot 60 Running Finger Roll Shot
1 Made Shot 61 Driving Finger Roll Shot
1 Made Shot 62 Turna

In [433]:
#Hardcore the scoring values of points
for action in EventScoringValues[1]:
    EventScoringValues[1][action] = 2 # i think some of these will be to be 3's
    
for action in EventScoringValues[3]:
    # are all free throw worth 1? technical?
    EventScoringValues[3][action] = 1
    
#all others 0

In [434]:
for action in EventPossChange[1]:
    EventPossChange[1][action] = True

#for action in EventPossChange[2]:
    #Instead of checking who has next possession on rebounds, check for every missed shot 
    #(Play by play will have 'false rebounds', for example after a missed ft that isn't the last one)
    
    #Sometimes have missed shot - rebound (out of bounds) - sub; 
    #NBA doesn't count sub as an extra possession because time not elapsed; might be fixed by only checking rebounds
    #since all missed shots have a rebound (except at EOP) - what about missed ft - rebound - ft; I think it might be fine
    #because 2nd ft should also trigger change in possession (unless it is missed which will lead to same possession or change 
    #that rebound check will trigger)
    #EventPossChange[2][action] = True

    
free_throws = EventCodes['Event_Msg_Type'] == 3
for action in EventPossChange[3]:
    #if action in (10, 12, 15):
        #Only potential change of possession on last shot of normal fts (not techincal, clear path, or flagrant)
        #Even if ft is made, I believe using the next poss logic should work, but may have to split into made vs missed
        #EventPossChange[3][action] = True
    #Changing such that all free throws possession is checked because of when tech fts happen it counts as a change of possession
    #(as long as time elapsed) Just need to make sure that for a given time step only one possession is added
    EventPossChange[3][action] = True
for action in EventPossChange[4]:
    EventPossChange[4][action] = True # all rebounds have to be checked
    
    
for action in EventPossChange[5]:
    EventPossChange[5][action] = True
    
#for action in EventPossChange[10]:
#    EventPossChange[10][action] = None # same with jump balls
    
#for action in EventPossChange[12]:
#    EventPossChange[12][action] = None # same with start and end of periods
           
#for action in EventPossChange[13]:
#    EventPossChange[13][action] = None 

In [435]:
for action in EventSub[8]:
    EventSub[8][action] = True
    
for action in EventPossChange[12]:
    EventSub[12][action] = True

### Order of operations
For Game in Games:

    For possession in Game:
        for player in possession:
            update scoring
    normalize by count

In [436]:
ID_OF_INTEREST = 'bfef77a3e57907855444410d490e7bfd'

In [437]:
class Game(object):
    
    def __init__(self, game_id, game_lineup, play_by_play):
        
        self.game_id = game_id
        self.play_by_play = self.chunk_pbp(play_by_play[play_by_play['Game_id'] == game_id])
        
        # tuple of lists of each player on the team
        self.teams = self.make_teams(game_lineup[game_lineup['Game_id']==game_id])
        self.poss_team_0 = None
        self.poss_change = None
        
        self.lineup_by_period =\
        [game_lineup[np.logical_and(game_lineup['Game_id']==game_id, \
                                    game_lineup['Period'] == i)] for i in xrange(1,5)]
    
    def score_event(self, idx, event):

        for play in event.itertuples():
            play_e, play_a = play_e, play_a = getattr(play, 'Event_Msg_Type'), getattr(play, 'Action_Type') #play[2], play[6]
            play_value = EventScoringValues[play_e][play_a]
            
            if play_value == 0:
                continue
            
            #Super hacky and breaks purpose of separating logic
            #but might be way to properly address tech fts counting as possessions
            if (getattr(play, 'Person1') in self.teams[0]) != self.poss_team_0:
                self._add_possessions()
                self.incr_poss()
                self.poss_team_0 = not self.poss_team_0
                
            play_value = getattr(play, 'Option1') #play[7]
            if (play_e == 3) and (play_value != 1):
                #On free throws, only a made ft if play_value ('Option1' = 1), it is sometimes 2 or another non-zero number
                continue

            for team, offense_or_defense in zip(self.teams, (self.poss_team_0, not self.poss_team_0)):
                for player in team.itervalues():
                    if player.active:
                        if offense_or_defense: #team 0 on offense?
                            player.off_points+=play_value
                        else:
                            player.def_points+=play_value
                        
                        
        
    def possession_change(self, idx,event):
        if self._EoP_in_event(event):
            self._add_possessions()
            self.poss_team_0 = None
            return
        #if idx in (36, 37):
        #    print event
        
        for play in event.itertuples():
            play_e, play_a = getattr(play, 'Event_Msg_Type'), getattr(play, 'Action_Type') #play[2], play[6]
            #poss_change = EventPossChange[play_e][play_a]
            self.poss_change = EventPossChange[play_e][play_a]
            if self.poss_change:
                next_poss_team_0 = self._get_next_poss(idx)
                #get_next_poss would return None if it can't find a valid next possession
                #I think this is only useful when debugging by looking parts of a quarter
                if self.poss_team_0 != next_poss_team_0 and next_poss_team_0 !=None:
                    self._add_possessions()
                #self.poss_team_0 = next_poss_team_0
                break
                
        #if self._EoP_in_event(event):
        #    self.poss_team_0 = None
                
    def _EoP_in_event(self, event):
        #Check PC time = 0
        return event.iloc[0, 5] == 0
       
    def _get_next_poss(self, idx):
        #lift the doPossessionFix from winston's codce
        if self._EoP_in_event(self.play_by_play[idx]):
            return not self.poss_team_0
        
        for event in self.play_by_play[idx+1:]:
            for play in event.itertuples():
                # gonna have to check that this makes sense...
                play_e, play_a = getattr(play, 'Event_Msg_Type'), getattr(play, 'Action_Type') #play[2], play[6]

                #Only made shot, missed shot, free throw, turnover, and end of period establishes posession
                if play_e in (1, 2, 3, 5, 13):
                    #may need to check on free throw (event 3) that it doesn't correspond to No Shot (action type 0)
                    
                    #if next time step has end of period, return opposite of current possession so add_possessions gets run
                    #Change -- trust that end of period team is correct, should make sure this is true
                    #Reason for change is that made shot -> end of period would work correctly, 
                    #but missed shot -> off rebound -> end of period mistakenly adds two possessions instead of one
                    if play_e == 13:
                        #return not self.poss_team_0
                        #Check whether end of period team is the same as a player on team 0's team ID
                        return getattr(play, 'Team_id') == self.teams[0].values()[0].team_id
                    if (play_e == 3) and (play_a not in (10, 11, 12, 13, 14, 15)):
                        #Only consider normal free throws (tech/flagrant fts can happen regardless of possession)
                        continue
                    return getattr(play, 'Person1') in self.teams[0] #play[11] in self.teams[0]
            
    def _add_possessions(self):
        
            for team, offense_or_defense in zip(self.teams, (self.poss_team_0, not self.poss_team_0)):
                for player in team.itervalues():
                    #Check that player is both active and has not already had a possession added in this time step
                    if player.active and not (player.incr_def_poss or player.incr_off_poss):
                        if offense_or_defense: #team 0 on offense?
                            player.incr_off_poss=True
                        else:
                            player.incr_def_poss=True
                            
                            
    def substitution(self,idx, event):

        for play in event.itertuples():
            play_e, play_a = getattr(play, 'Event_Msg_Type'), getattr(play, 'Action_Type') #play[2], play[6]
            sub = EventSub[play_e][play_a]
            
            if sub:
                if play_e == 8: #Garden variety substitution
                    outgoing_player, ingoing_player = getattr(play, 'Person1'), getattr(play, 'Person2') #play[11], play[12]
                    if ingoing_player == ID_OF_INTEREST:
                            print 'Spicy P in', idx
                    if outgoing_player == ID_OF_INTEREST:
                            print 'Spicy P out', idx
                            
                    
                    if outgoing_player in self.teams[0]:
                        self.teams[0][outgoing_player].active = False
                        self.teams[0][ingoing_player].active = True
                        
                        #an outgoing player's posession ends, unless it would otherwise be counted
                        if self.poss_team_0:
                            self.teams[0][outgoing_player].incr_off_poss =True
                        else:
                            self.teams[0][outgoing_player].incr_def_poss =True

                    else:
                        self.teams[1][outgoing_player].active = False
                        self.teams[1][ingoing_player].active = True
                        
                        if self.poss_team_0:
                            self.teams[1][outgoing_player].incr_def_poss =True
                        else:
                            self.teams[1][outgoing_player].incr_off_poss =True

                        
                else: #start of period
                    period = getattr(play, 'Period')-1 #play[3]-1
                    if period == 0:
                        continue # we handle this on startup
                        
                    for team in self.teams:
                        for player in team.itervalues():
                            player.active = False # turn everyone off
                    
                    period_lineup = self.lineup_by_period[period]
                    for _, row in period_lineup.iterrows():
                        player_id = row[2]
                        
                        active = row[4] == 'A'
                        if player_id in self.teams[0]:
                            self.teams[0][player_id].active = active
                        else:
                            self.teams[1][player_id].active = active

    
    def chunk_pbp(self, game_pbp):
        sorted_pbp = self.sort_pbp(game_pbp)
        real_time = sorted_pbp['PC_Time'].as_matrix()+  (4-sorted_pbp['Period'].as_matrix())*7200
        _, unique_idxs = np.unique(real_time, return_index=True)
        return np.split(game_pbp, indices_or_sections=unique_idxs[::-1] )[1:] # remove the first one thats empty
        
    def sort_pbp(self, game_pbp):
        return game_pbp.sort_values(['Period', 'PC_Time', 'WC_Time', 'Event_Num'],\
                                    ascending=[True, False, True,True])
    
    def make_teams(self, game_lineup):
        team1_id, team2_id = np.unique(game_lineup['Team_id'])
        team1, team2 = {},{}
        for team_id, team_arr in izip((team1_id, team2_id), (team1, team2)):
            players_on_team = game_lineup[game_lineup['Team_id']==team_id]['Person_id'].unique()
            for player in players_on_team:

                player_data = game_lineup[game_lineup['Person_id'] == player] #ignore possessions
                active_to_start = 1 in set(player_data['Period'])
                team_arr[player] = Player(player_data.iloc[0], active_to_start)
                
        return team1, team2
    
    def incr_poss(self):
        """
        Increment possession counters and resent them
        """
        for team in self.teams:
            for player in team.itervalues(): 
                if player.incr_off_poss:
                    player.off_poss+=1
                if player.incr_def_poss:
                    player.def_poss+=1
                player.incr_off_poss=False
                player.incr_def_poss=False
    
    def compute_ratings(self, up_to_period = 4):
        
        for idx, event in enumerate(self.play_by_play):
            if self.poss_team_0 == None:
                #At start of periods, don't know who has possession so use get_next_poss method to establish
                self.poss_team_0 = self._get_next_poss(idx)
            
            self.score_event(idx, event)
            self.possession_change(idx, event)
            self.substitution(idx, event)
            self.incr_poss()
            #Moved the following from possession_change to here. This at least worked for Q1, but may break other stuff.
            #Reason is when subbed out on a ft, the possession change function would flip possession and then substitution method
            #would add possession to the opposite of possession change (offense/defense). Moving substitution before possession_change
            #not possible because want treat players on court during fts before subbing them.
            #Normally this is only done when poss_change == True, but might be valid to do on every iteration:
            #get_next_poss essentially tells you who has the ball except in the cases where the current time step is one of the
            #poss_change == True events (made, missed shot, ft, turnover)
            if self.poss_change:
                self.poss_team_0 = self._get_next_poss(idx)
                self.poss_change = None
 
            for team in self.teams:
                for player in team.itervalues():
                    if player.player_id == ID_OF_INTEREST:
                        print player.off_points, player.def_points, player.off_poss, player.def_poss
                        pass
            if self._EoP_in_event(event):
                if event.iloc[0,3] == up_to_period: #event[0][3] == up_to_period:
                    print 'End Of Period', idx
                    break
            for teamno, team in enumerate(self.teams):
                active_players = sum(int(player.active) for player in team.itervalues())
                assert active_players == 5; "Team %d has %d players active"%(teamno+1, active_players)
        
        gameDict = {}
        for team in self.teams:
            for player in team.itervalues():
                player.finalize_ratings()
                #print player.player_id, player.off_rating, player.def_rating
                gameDict[player.player_id] = np.array([player.off_points, player.def_points, 
                                                       player.off_poss, player.def_poss, 
                                                       round(player.off_rating, 1), round(player.def_rating,1)])
            print '*'*20
        return gameDict

In [438]:
class Player(object):
    
    def __init__(self, player_data, active_to_start):
        self.player_id = player_data['Person_id']
        self.team_id = player_data['Team_id']
        self.active = active_to_start
        
        self.off_points = 0
        self.def_points = 0
        
        self.off_poss = 0
        self.def_poss = 0
        
        self.incr_off_poss = False
        self.incr_def_poss = False
        
        self.off_rating = np.nan
        self.def_rating = np.nan
        
    def finalize_ratings(self):
        #print (self.off_points, self.def_points, self.off_poss, self.def_poss)
        if self.off_poss>0:
            self.off_rating = self.off_points*100.0/self.off_poss
        if self.def_poss>0:
            self.def_rating = self.def_points*100.0/self.def_poss

In [399]:
game0_id = GameLineup.iloc[0]['Game_id']
game0 = Game(game0_id, GameLineup, PlayByPlay)

C:\Users\Winston\Anaconda3\envs\ipykernel_py2\lib\site-packages\ipykernel_launcher.py:168: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [18]:
game0_id

'006728e4c10e957011e1f24878e6054a'

In [19]:
for event in game0.play_by_play[57:87]:
    for play in event:
        play_e, play_a = play[2], play[6]
        x= EventCodes[np.logical_and(EventCodes['Event_Msg_Type'] == play_e, EventCodes['Action_Type']==play_a)]
        print str(x['Event_Msg_Type_Description']), str(x['Action_Type_Description'])
        print

260    Timeout
Name: Event_Msg_Type_Description, dtype: object 260    Regular
Name: Action_Type_Description, dtype: object

254    Substitution
Name: Event_Msg_Type_Description, dtype: object 254     
Name: Action_Type_Description, dtype: object

254    Substitution
Name: Event_Msg_Type_Description, dtype: object 254     
Name: Action_Type_Description, dtype: object

254    Substitution
Name: Event_Msg_Type_Description, dtype: object 254     
Name: Action_Type_Description, dtype: object

131    Missed Shot
Name: Event_Msg_Type_Description, dtype: object 131    Turnaround Fadeaway shot
Name: Action_Type_Description, dtype: object

175    Rebound
Name: Event_Msg_Type_Description, dtype: object 175    Unknown
Name: Action_Type_Description, dtype: object

106    Missed Shot
Name: Event_Msg_Type_Description, dtype: object 106    Turnaround Hook Shot
Name: Action_Type_Description, dtype: object

175    Rebound
Name: Event_Msg_Type_Description, dtype: object 175    Unknown
Name: Action_Type_D

In [20]:
ID_OF_INTEREST

'618f6d58ab2881152607c2a6e057bc51'

In [21]:
game0.compute_ratings()

Spicy P out 57
Spicy P in 112
Spicy P out 155
Spicy P out 197
Spicy P in 278
End Of Period 315
0
0
ed95dff5440fadf3042b5acacea81eed None None
75
72
766802a8fda500d7945950de7398c9c6 108.0 111.111111111
36
33
2ad626904c8b28cceb8e12c624a84240 94.4444444444 124.242424242
71
66
42e0d7167f04a4ff958c6442da0e6851 112.676056338 115.151515152
0
0
e59b921ab3da55f632bc748beb12805a None None
60
57
8d2127290c94bd41b82a2938734bc750 110.0 122.807017544
31
29
f4a5ca938177c407a9dab5412e39498f 90.3225806452 103.448275862
0
0
bd45fe7dba52aa2cd00ba80ff107d05b None None
0
0
dd1da128c27db468d95b99b583f8a57d None None
16
15
ae53f8ba6761b64a174051da817785bc 87.5 93.3333333333
50
47
618f6d58ab2881152607c2a6e057bc51 122.0 119.14893617
0
0
e816ff284dc3f965b8f3d605a3b91bae None None
81
78
c5dd5b2e3b975f0849d9b74e74125cb9 106.172839506 116.666666667
0
0
c10b49616a2f4a23607dc1a8be4fde9f None None
35
33
5db9c1c8184510fee8161e7fafdc9c49 85.7142857143 96.9696969697
********************
0
0
7027df5d9c51192f3527ac8c74b4d

In [371]:
def sortedPlayByPlay(PlayByPlay):
    """Returns PlayByPlay with nested sorted given by hackathon pdf file.
    
    Input:
        PlayByPlay: Pandas Dataframe from reading Play_by_Play csv file"""
    return PlayByPlay.sort_values(['Period', 'PC_Time', 'WC_Time', 'Event_Num'], ascending=[True, False, True,True])

In [372]:
def compareGameDicts(gameDict1, gameDict2):
    """Used to compare two gameDicts to see if there are any differences. 
        Typically used when making changes to gameRatings function to determine 
        if it changed any output. Assumes both gameDict have the same exact players (which they should).
    """
    
    for player in gameDict1.keys():
        if not (gameDict1[player] == gameDict2[player]).all():
            print player
            return False
    return True

In [373]:
game_id = np.unique(GameLineup['Game_id'])[21]

In [439]:
gamePlayByPlay = sortedPlayByPlay(PlayByPlay[PlayByPlay['Game_id'] == game_id])

In [440]:
game = Game(game_id, GameLineup, gamePlayByPlay[240:352])
gameDict = game.compute_ratings()
gameDict

C:\Users\Winston\Anaconda3\envs\ipykernel_py2\lib\site-packages\ipykernel_launcher.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0 0 0 0
0 0 0 0
0 0 0 1
0 0 0 1
0 0 0 1
0 0 0 1
2 0 1 1
2 0 1 2
4 0 2 2
4 0 2 2
4 0 2 2
4 0 2 3
4 0 2 3
6 0 3 3
6 0 3 3
6 2 3 4
6 2 3 4
6 2 4 4
6 2 4 4
6 4 4 5
6 4 4 5
6 4 4 5
8 4 5 5
8 4 5 6
8 4 5 6
8 4 6 6
8 4 6 6
8 4 6 6
8 4 6 6
8 4 6 7
8 4 6 7
8 4 7 7
8 6 7 8
10 6 8 8
10 6 8 8
Spicy P out 35
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
10 6 8 9
********************
********************


{'0b978fcfa7f2ec839c563a755e345ff8': array([ 0.,  0.,  0.,  0., nan, nan]),
 '14de632b07100527b0ced12fd4eeffb7': array([ 0.,  0.,  0.,  0., nan, nan]),
 '18a823379f2bf4e4be8b419698cde91c': array([ 0.,  0.,  0.,  0., nan, nan]),
 '1a6703883f8f47bb4daf09c03be3bda2': array([ 20. ,  11. ,  16. ,  16. , 125. ,  68.8]),
 '1dabb767e07d0aa702ee58d41c15eab1': array([  7. ,  17. ,  13. ,  14. ,  53.8, 121.4]),
 '255fe2a8be0ed5c06dd99969ab4fea55': array([ 0.,  0.,  0.,  0., nan, nan]),
 '31598ba01a3fff03ed0a87d7dea11dfe': array([ 25. ,  13. ,  21. ,  21. , 119. ,  61.9]),
 '32c044aa84d75ccd78c3c9f2aeb33bd9': array([0., 0., 2., 2., 0., 0.]),
 '3626b893fc73a5cbd67d1ea48a5c7039': array([ 25. ,  13. ,  21. ,  21. , 119. ,  61.9]),
 '36fdadf436b164ee29174c8e1fde7271': array([  7. ,  11. ,   9. ,  10. ,  77.8, 110. ]),
 '3d75035d20b173a867d4bf32c8a58f0b': array([ 15. ,   7. ,  13. ,  12. , 115.4,  58.3]),
 '52c6125836c465f4ac5232121dacb49d': array([  5.,   2.,   5.,   5., 100.,  40.]),
 '619d3e44dc84b3

In [413]:
game = Game(game_id, GameLineup, gamePlayByPlay[240:352])
gameDict2 = game.compute_ratings()
gameDict2

C:\Users\Winston\Anaconda3\envs\ipykernel_py2\lib\site-packages\ipykernel_launcher.py:168: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0 0 0 0
0 0 0 1
0 0 0 1
0 0 0 1
0 0 0 1
0 0 0 1
2 0 1 1
2 0 1 2
4 0 2 2
4 0 2 2
4 0 2 3
4 0 2 3
4 0 2 3
6 0 3 3
6 0 3 3
6 2 3 4
6 2 4 4
6 2 4 4
6 2 4 4
6 4 4 5
6 4 4 5
6 4 4 5
8 4 5 5
8 4 5 6
8 4 6 6
8 4 6 6
8 4 6 6
8 4 6 6
8 4 6 7
8 4 6 7
8 4 7 7
8 4 7 7
8 6 7 8
10 6 8 8
10 6 8 9
10 6 8 9
10 6 9 9
10 6 9 9
10 6 9 10
10 6 9 10
11 6 10 10
11 6 10 11
14 6 11 11
14 6 11 12
14 6 12 12
14 8 12 13
16 8 13 13
16 11 13 14
16 11 14 14
16 11 14 14
16 11 14 15
16 11 14 15
16 11 14 15
16 11 14 15
18 11 15 15
18 11 15 16
18 11 15 16
Spicy P out 57
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
20 11 16 16
********************
********************


{'0b978fcfa7f2ec839c563a755e345ff8': array([ 0.,  0.,  0.,  0., nan, nan]),
 '14de632b07100527b0ced12fd4eeffb7': array([ 0.,  0.,  0.,  0., nan, nan]),
 '18a823379f2bf4e4be8b419698cde91c': array([ 0.,  0.,  0.,  0., nan, nan]),
 '1a6703883f8f47bb4daf09c03be3bda2': array([ 20. ,  11. ,  16. ,  16. , 125. ,  68.8]),
 '1dabb767e07d0aa702ee58d41c15eab1': array([  7. ,  17. ,  13. ,  14. ,  53.8, 121.4]),
 '255fe2a8be0ed5c06dd99969ab4fea55': array([ 0.,  0.,  0.,  0., nan, nan]),
 '31598ba01a3fff03ed0a87d7dea11dfe': array([ 25. ,  13. ,  21. ,  21. , 119. ,  61.9]),
 '32c044aa84d75ccd78c3c9f2aeb33bd9': array([0., 0., 2., 2., 0., 0.]),
 '3626b893fc73a5cbd67d1ea48a5c7039': array([ 25. ,  13. ,  21. ,  21. , 119. ,  61.9]),
 '36fdadf436b164ee29174c8e1fde7271': array([  7. ,  11. ,   9. ,  10. ,  77.8, 110. ]),
 '3d75035d20b173a867d4bf32c8a58f0b': array([ 15. ,   7. ,  13. ,  12. , 115.4,  58.3]),
 '52c6125836c465f4ac5232121dacb49d': array([  5.,   2.,   5.,   5., 100.,  40.]),
 '619d3e44dc84b3

In [257]:
array = np.array
winstonGame21Dict = {'0b978fcfa7f2ec839c563a755e345ff8': array([8, 4, 6, 5]),
 '14de632b07100527b0ced12fd4eeffb7': array([0, 0, 0, 0]),
 '18a823379f2bf4e4be8b419698cde91c': array([0, 0, 0, 0]),
 '1a6703883f8f47bb4daf09c03be3bda2': array([26, 21, 20, 20]),
 '1dabb767e07d0aa702ee58d41c15eab1': array([12, 10,  9,  9]),
 '255fe2a8be0ed5c06dd99969ab4fea55': array([0, 0, 0, 0]),
 '31598ba01a3fff03ed0a87d7dea11dfe': array([13,  5,  9,  9]),
 '32c044aa84d75ccd78c3c9f2aeb33bd9': array([2, 5, 4, 5]),
 '3626b893fc73a5cbd67d1ea48a5c7039': array([34, 25, 24, 24]),
 '36fdadf436b164ee29174c8e1fde7271': array([12, 10,  9,  9]),
 '3d75035d20b173a867d4bf32c8a58f0b': array([12, 12, 11, 10]),
 '52c6125836c465f4ac5232121dacb49d': array([8, 4, 6, 5]),
 '619d3e44dc84b366bd685de3e94b3bec': array([0, 0, 0, 0]),
 '6f6a807d57aae8f651222523dc82dc35': array([0, 0, 0, 0]),
 '722a380c9b59ef42226e8d392824dcb9': array([13, 24, 16, 16]),
 '7f438c18058290903c46dfe9d71bd68a': array([23, 29, 20, 20]),
 '821887f9a002be16b5f79729fae59e01': array([0, 0, 0, 0]),
 '8c7a7249d80b1489594b3a2a87f3f19d': array([0, 0, 0, 0]),
 '942a84f05f4ab956125f68ec0963481f': array([0, 0, 1, 1]),
 '94e99d76e87ee926faab66d382b3a955': array([0, 0, 0, 0]),
 '95920e4bf5b6c15ba8dffbf959b38ba5': array([25, 34, 24, 25]),
 'a1591595c04d12e88e3cb427fb667618': array([26, 21, 19, 19]),
 'bfef77a3e57907855444410d490e7bfd': array([22, 13, 15, 14]),
 'c950aaad2e56c87e9ac7281016d37cb6': array([0, 0, 0, 0]),
 'e49b2cc3f9aacd500b11a35b1c57112d': array([0, 0, 0, 0]),
 'ef8b068ab7ac9d387b256404acd24cd5': array([13, 24, 16, 16]),
 'f3d6e924c2736c9e5771b0784e31c2df': array([0, 0, 0, 0]),
 'fb64ca4b8beaf4c4c6e4575fe2f3abd7': array([25, 34, 24, 24]),
 'fbcda0bcb861e4726ca8871b8965ede4': array([0, 0, 0, 0]),
 'ff59dc439c6c323320bc355afe884fcb': array([21, 20, 16, 16])}

In [241]:
compareGameDicts(game0Dict, winstonGame0Dict)

True

In [46]:
game0Dict

{'1b37fed4823f31ccbf03515f4b9fe438': array([0, 0, 0, 0]),
 '2ad626904c8b28cceb8e12c624a84240': array([8, 4, 6, 6], dtype=int64),
 '42e0d7167f04a4ff958c6442da0e6851': array([20, 13, 14, 13], dtype=int64),
 '4413c19cf092bda39332d7833c90bfe6': array([5, 6, 7, 7], dtype=int64),
 '44230324724c84f122ac62a5f0918314': array([4, 8, 8, 8], dtype=int64),
 '48ec4e6c52f418d5ca4ef510ba473ea0': array([16, 24, 19, 19], dtype=int64),
 '4dd3d6a51dc97c651d3a86eec4362a1f': array([0, 0, 0, 0]),
 '4ef3dae16c436459ff05156abca5cebd': array([4, 6, 5, 4], dtype=int64),
 '5cce6ffa455e6372d9de0de400482ab6': array([4, 8, 6, 6], dtype=int64),
 '5db9c1c8184510fee8161e7fafdc9c49': array([8, 4, 8, 8], dtype=int64),
 '616281dee946056b071699476fdee9ec': array([16, 22, 17, 18], dtype=int64),
 '618f6d58ab2881152607c2a6e057bc51': array([22, 16, 16, 15], dtype=int64),
 '7027df5d9c51192f3527ac8c74b4d08d': array([0, 0, 0, 0]),
 '766802a8fda500d7945950de7398c9c6': array([30, 20, 23, 23], dtype=int64),
 '8d2127290c94bd41b82a293

In [209]:
game0.play_by_play[0]

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type
0,006728e4c10e957011e1f24878e6054a,2,12,1,546410,7200,0,0,0,0,1473d70e5646a26de3c52aa1abd85b1f,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0,0,0,0
1,006728e4c10e957011e1f24878e6054a,4,10,1,546420,7200,0,0,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,8d2127290c94bd41b82a2938734bc750,99104de2626f67c1fa2ce70504970c3f,766802a8fda500d7945950de7398c9c6,2,4,5,4


In [236]:
game0.teams[0].values()[0].team_id

'01be0ad4af7aeb1f6d2cc2b6b6d6d811'

In [247]:
GameLineup.iloc[26]['Game_id']

'006728e4c10e957011e1f24878e6054a'

In [250]:
np.unique(GameLineup['Game_id'])[21]

'58f4e85cad83adedfa96749b0cde2564'

In [355]:
round(100*21.0/16, 1)

131.3